In [1]:
# maze test
import matplotlib
matplotlib.use('TkAgg')
import numpy as np
import tensorflow as tf
import gym
import matplotlib.pyplot as plt

In [2]:
from generator_maze import *

In [3]:
env = random_maze(width=32, height=32, complexity=0.8, density=0.5, render_trace=True)
env.reset()
env.render()

<Figure size 640x480 with 2 Axes>

In [4]:
INPUTSIZE = env.observation_space.shape
OUTPUTSIZE = env.action_space.n
DISCOUNT = 0.9
REPLAY = 5000
BATCH_SIZE = 32
TARGET_UPDATE_FREQUENCY = 5
MAX_EPISODE = 100
print(INPUTSIZE, OUTPUTSIZE)

(33, 33) 4


In [5]:
def get_copy_params(src:str, dest:str):
    holder = []
    src_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, socpe = src)
    dest_Vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = dest)
    for src_var, dest_var in zip(src_vars, dest_vars):
        holder.append(dest_var.assign(src_var.value()))
    return holder

In [6]:
def replay_train(mainDQN, targetDQN, train_batch):
    states = np.vstack([x[0] for x in train_batch])
    actions = np.array([x[1] for x in train_batch])
    rewards = np.array([x[2] for x in train_batch])
    next_states = np.vstack([x[3] for x in train_batch])
    done = np.array([x[4] for x in train_batch])
    
    Q_target = reward + DISCOUNT*np.max(tratgetDQN.predict(next_states), axis=1)*~done
    X = states
    y = mainDQN.predict(states)
    y[np.arange(len(states)), actions] = Q_target
    
    return mainDQN.update(X, y)
    